## DNN Frontier 예측 모델

### modules import

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_file, plot_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

### 데이터 로드

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# g = [100, 500, 1000, 5000, 10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000] # DMU
# h = [13,14,15,22,23,24,25,31,32,33,34,35,41,42,43,44,45,51,52,53,54,55] # Output & Input
# i = [1,2] # Orientation
# j = [1,2,3] # Card Range

g = [640000] # DMU
h = [41,42,43,44,45,51,52,53,54,55] # Output & Input 
i = [1,2] # Orientation
j = [1,2,3] # Card Range  # {1: 0.1}, {2: 0.05},{3: 0.2}
for a in g:
  for b in h:
    for c in i:
      for d in j:
        ####

        if b == 41 and c == 1:
          continue

        ####
        r_1 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/dataset_before_categorized/{}_{}_{}_dat.csv'.format(a,b,c) # total dataset
        r_2 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/test_dataset/test_df/{}_{}_{}_test_dat.csv'.format(a,b,c) # test dataset which outlier is removed

        w_1 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/train_dataset/train_categorized_df/Cat_{}_{}_{}_{}_train_dat.csv'.format(a,b,c,d) # categorized train data
        w_2 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/test_dataset/test_categorized_df/Cat_{}_{}_{}_{}_test_dat.csv'.format(a,b,c,d) # categorized test data
        w_3 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/pred_df/Cat_{}_{}_{}_{}_Pred_df.csv'.format(a,b,c,d) # prediction data
        w_4 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/visualization_and_history/Cat_{}_{}_{}_{}_scatter.png'.format(a,b,c,d) # scattor plot
        w_5 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/visualization_and_history/Cat_{}_{}_{}_{}_error_histogram.png'.format(a,b,c,d) # error histogram
        w_6 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/visualization_and_history/Cat_{}_{}_{}_{}_epoch_history.csv'.format(a,b,c,d) # epoch history
        w_7 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/accuracy_txt/Cat_{}_{}_{}_{}_accuracy.txt'.format(a,b,c,d) # accuracy
        

        raw_dataset = pd.read_csv(r_1, encoding = 'UTF-8')
        dataset = raw_dataset.copy()

        raw_test_dataset = pd.read_csv(r_2, encoding = 'UTF-8')
        test_dataset = raw_test_dataset.copy()
        test_dataset = test_dataset.iloc[:,1:]

        if d == 1:
          bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
          lab = ['J','I','H','G','F','E','D','C','B','A']
        elif d == 2:
          bins = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
          lab = ['T','S','R','Q','P','O','N','M','L','K','J','I','H','G','F','E','D','C','B','A']
        else:
          bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
          lab = ['E','D','C','B','A']

        target_dat = dataset.iloc[:,int(b//10+b%10)]
        dataset['eff_lev'] = pd.cut(target_dat, bins=bins, labels = lab)
        dataset.to_csv(w_1, encoding = 'UTF-8')

        test_target_dat = test_dataset.iloc[:,int(b//10+b%10)]
        test_dataset['eff_lev'] = pd.cut(test_target_dat, bins=bins, labels = lab)
        test_dataset.to_csv(w_2, encoding = 'UTF-8')

        dataset = dataset.loc[0:int(0.8*a-1)]

        X = dataset.iloc[:, 0:int(b//10+b%10)]
        LabE = LabelEncoder()
        Y_obj = dataset['eff_lev']
        LabE.fit(Y_obj)
        Y = LabE.transform(Y_obj)
        X['eff_lev'] = Y
        train_dataset = X

        X_t = test_dataset.iloc[:, 0:int(b//10+b%10)]
        LabE = LabelEncoder()
        Y_obj_t = test_dataset['eff_lev']
        LabE.fit(Y_obj_t)
        Y_t = LabE.transform(Y_obj_t)
        X_t['eff_lev'] = Y_t
        test_dataset = X_t.squeeze()

        train_dataset = train_dataset.squeeze()

        ################################################################################

        train_labels = train_dataset.iloc[:,-1]
        train_dataset = train_dataset.iloc[:,0:-1]
        test_labels = test_dataset.iloc[:,-1]
        test_dataset = test_dataset.iloc[:,0:-1]

        train_stats = train_dataset.describe()
        train_stats = train_stats.transpose()

        train_dataset = train_dataset.astype(float)
        test_dataset = test_dataset.astype(float)

        def normalization(x):
          return (x -  train_stats['mean']) / train_stats['std']

        normed_train_data = normalization(train_dataset)
        normed_test_data = normalization(test_dataset)

        def create_model():
          
          model = keras.Sequential([
              keras.layers.Dense(128, activation = 'relu'),
              keras.layers.Dense(64, activation = 'relu'),
              keras.layers.Dense(64, activation = 'relu'),
              keras.layers.Dense(len(train_labels), activation = 'softmax') 
          ])

          model.compile(optimizer = 'adam',
                        loss = 'sparse_categorical_crossentropy',
                        metrics = ['accuracy'])

          return model

        model = create_model()
        sample_batch = normed_train_data[:256]
        sample_result = model.predict(sample_batch)

        epochs = 30

        model = create_model()

        early_stop = EarlyStopping(monitor = 'val_loss', patience = 2)

        history = model.fit(normed_train_data, train_labels, epochs = epochs,
                            validation_split = 0.2, callbacks = [early_stop])
        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch

        ################################################################################
        #######  Result Part  ##########################################################
        ################################################################################

        test_pred_proba = model.predict(normed_test_data.iloc[0:1,:])
        test_pred = test_pred_proba.argmax(axis = -1)

        for q_0 in range(1,int((a*0.2*0.9)//100)+1):
          test_pred_proba = model.predict(normed_test_data.iloc[1+100*(q_0-1):q_0*100+1,:])
          test_pred_sample = test_pred_proba.argmax(axis = -1)
          test_pred= np.concatenate((test_pred, test_pred_sample), axis = 0)


        df = pd.DataFrame(test_pred, columns = ['pred'])
        df.index = list(range(0,int(a*0.2*0.9)))
        label_data = pd.Series.to_frame(test_labels)
        df_pred = pd.concat([label_data, df],axis = 1)
        df_pred.to_csv(w_3, encoding = 'UTF-8')

        ################################################################################

        plt.cla()
        plt.clf()
        plt.scatter(test_labels, test_pred, s= 1 )
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.axis('equal')
        plt.axis('square')
        plt.grid()
        plt.xlim([0, 10])
        plt.ylim([0, 10])
        plt.plot([-1000, 1000], [-1000, 1000]);
        plt.savefig(w_4)

        ################################################################################
        if d ==1:
          plt.cla()
          plt.clf()
          error = test_pred - test_labels
          plt.hist(error, bins = 20, range = [-9,9])
          plt.xlabel("Prediction Error")
          plt.grid()
          plt.ylabel("Count")
          plt.savefig(w_5)

        elif d == 2:
          plt.cla()
          plt.clf()
          error = test_pred - test_labels
          plt.hist(error, bins = 40, range = [-19,19])
          plt.xlabel("Prediction Error")
          plt.grid()
          plt.ylabel("Count")
          plt.savefig(w_5)
        
        else:
          plt.cla()
          plt.clf()
          error = test_pred - test_labels
          plt.hist(error, bins = 40, range = [-4,4])
          plt.xlabel("Prediction Error")
          plt.grid()
          plt.ylabel("Count")
          plt.savefig(w_5)

        ################################################################################

        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch
        hist.to_csv(w_6, encoding = 'UTF-8')

        ################################################################################

        test_loss, test_acc = model.evaluate(normed_test_data,  test_labels, verbose=2)
        test_acc_str = str(test_acc)

        f = open(w_7,'a')
        f.write(test_acc_str)
        f.close()

        ################################################################################

8/8 [==============================] - 2s 5ms/step
Epoch 1/30
12800/12800 [==============================] - 781s 61ms/step - loss: 1.3757 - accuracy: 0.4495 - val_loss: 1.2735 - val_accuracy: 0.4816
Epoch 2/30
12800/12800 [==============================] - 780s 61ms/step - loss: 1.2670 - accuracy: 0.4866 - val_loss: 1.2555 - val_accuracy: 0.4906
Epoch 3/30
12800/12800 [==============================] - 777s 61ms/step - loss: 1.2504 - accuracy: 0.4949 - val_loss: 1.2413 - val_accuracy: 0.4987
Epoch 4/30
12800/12800 [==============================] - 779s 61ms/step - loss: 1.2424 - accuracy: 0.4974 - val_loss: 1.2366 - val_accuracy: 0.4990
Epoch 5/30
12800/12800 [==============================] - 778s 61ms/step - loss: 1.2375 - accuracy: 0.4998 - val_loss: 1.2337 - val_accuracy: 0.5016
Epoch 6/30
12800/12800 [==============================] - 778s 61ms/step - loss: 1.2346 - accuracy: 0.5002 - val_loss: 1.2385 - val_accuracy: 0.4986
Epoch 7/30
4/4 [==============================] - 0s 4m

In [ ]:
test_loss, test_acc = model.evaluate(normed_test_data,  test_labels, verbose=2)
test_acc

7200/7200 - 972s - loss: 1.3647 - accuracy: 0.4278 - 972s/epoch - 135ms/step


0.4278397858142853

In [ ]:
# g = [640000, 1280000] # DMU
# h = [13,14,15,22,23,24,25,31,32,33,34,35,41,42,43,44,45,51,52,53,54,55] # Output & Input
# i = [1,2] # Orientation
# j = [1,2,3] # Card Range
#,41,42,43,44,45,51,52,53,54,55]
g = [640000] # DMU
h = [41,42,43,44,45,51,52,53,54,55] # Output & Input 
i = [1,2] # Orientation
j = [1,2,3] # Card Range  # {1: 0.1}, {2: 0.05},{3: 0.2}
for a in g:
  for b in h:
    for c in i:
      for d in j:
        ####

        if b == 41 and c == 1:
          continue

        ####
        r_1 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/dataset_before_categorized/{}_{}_{}_dat.csv'.format(a,b,c) # total dataset
        r_2 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/test_dataset/test_df/{}_{}_{}_test_dat.csv'.format(a,b,c) # test dataset which outlier is removed

        w_1 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/train_dataset/train_categorized_df/Cat_{}_{}_{}_{}_train_dat.csv'.format(a,b,c,d) # categorized train data
        w_2 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/test_dataset/test_categorized_df/Cat_{}_{}_{}_{}_test_dat.csv'.format(a,b,c,d) # categorized test data
        w_3 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/pred_df/Cat_{}_{}_{}_{}_Pred_df.csv'.format(a,b,c,d) # prediction data
        w_4 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/visualization_and_history/Cat_{}_{}_{}_{}_scatter.png'.format(a,b,c,d) # scattor plot
        w_5 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/visualization_and_history/Cat_{}_{}_{}_{}_error_histogram.png'.format(a,b,c,d) # error histogram
        w_6 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/visualization_and_history/Cat_{}_{}_{}_{}_epoch_history.csv'.format(a,b,c,d) # epoch history
        w_7 = '/content/gdrive/MyDrive/DNN_Frontier/0902_data/re_DNN_Frontier_Classification/accuracy_txt/Cat_{}_{}_{}_{}_accuracy.txt'.format(a,b,c,d) # accuracy
        

        raw_dataset = pd.read_csv(r_1, encoding = 'UTF-8')
        dataset = raw_dataset.copy()

        raw_test_dataset = pd.read_csv(r_2, encoding = 'UTF-8')
        test_dataset = raw_test_dataset.copy()
        test_dataset = test_dataset.iloc[:,1:]

        if d == 1:
          bins = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
          lab = ['J','I','H','G','F','E','D','C','B','A']
        elif d == 2:
          bins = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
          lab = ['T','S','R','Q','P','O','N','M','L','K','J','I','H','G','F','E','D','C','B','A']
        else:
          bins = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]
          lab = ['E','D','C','B','A']

        target_dat = dataset.iloc[:,int(b//10+b%10)]
        dataset['eff_lev'] = pd.cut(target_dat, bins=bins, labels = lab)
        dataset.to_csv(w_1, encoding = 'UTF-8')

        test_target_dat = test_dataset.iloc[:,int(b//10+b%10)]
        test_dataset['eff_lev'] = pd.cut(test_target_dat, bins=bins, labels = lab)
        test_dataset.to_csv(w_2, encoding = 'UTF-8')

        dataset = dataset.loc[0:int(0.8*a-1)]

        X = dataset.iloc[:, 0:int(b//10+b%10)]
        LabE = LabelEncoder()
        Y_obj = dataset['eff_lev']
        LabE.fit(Y_obj)
        Y = LabE.transform(Y_obj)
        X['eff_lev'] = Y
        train_dataset = X

        X_t = test_dataset.iloc[:, 0:int(b//10+b%10)]
        LabE = LabelEncoder()
        Y_obj_t = test_dataset['eff_lev']
        LabE.fit(Y_obj_t)
        Y_t = LabE.transform(Y_obj_t)
        X_t['eff_lev'] = Y_t
        test_dataset = X_t.squeeze()

        train_dataset = train_dataset.squeeze()

        ################################################################################

        train_labels = train_dataset.iloc[:,-1]
        train_dataset = train_dataset.iloc[:,0:-1]
        test_labels = test_dataset.iloc[:,-1]
        test_dataset = test_dataset.iloc[:,0:-1]

        train_stats = train_dataset.describe()
        train_stats = train_stats.transpose()

        train_dataset = train_dataset.astype(float)
        test_dataset = test_dataset.astype(float)

        def normalization(x):
          return (x -  train_stats['mean']) / train_stats['std']

        normed_train_data = normalization(train_dataset)
        normed_test_data = normalization(test_dataset)

        def create_model():
          
          model = keras.Sequential([
              keras.layers.Dense(128, activation = 'relu'),
              keras.layers.Dense(64, activation = 'relu'),
              keras.layers.Dense(64, activation = 'relu'),
              keras.layers.Dense(len(train_labels), activation = 'softmax') 
          ])

          model.compile(optimizer = 'adam',
                        loss = 'sparse_categorical_crossentropy',
                        metrics = ['accuracy'])

          return model

        model = create_model()
        sample_batch = normed_train_data[:256]
        sample_result = model.predict(sample_batch)

        epochs = 30

        model = create_model()

        early_stop = EarlyStopping(monitor = 'val_loss', patience = 2)

        history = model.fit(normed_train_data, train_labels, epochs = epochs,
                            validation_split = 0.2, callbacks = [early_stop])
        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch

        ################################################################################
        #######  Result Part  ##########################################################
        ################################################################################

        test_pred_proba = model.predict(normed_test_data.iloc[0:1,:])
        test_pred = test_pred_proba.argmax(axis = -1)

        for q_0 in range(1,int((a*0.2*0.9)//100)+1):
          test_pred_proba = model.predict(normed_test_data.iloc[1+100*(q_0-1):q_0*100+1,:])
          test_pred_sample = test_pred_proba.argmax(axis = -1)
          test_pred= np.concatenate((test_pred, test_pred_sample), axis = 0)


        df = pd.DataFrame(test_pred, columns = ['pred'])
        df.index = list(range(0,int(a*0.2*0.9)))
        label_data = pd.Series.to_frame(test_labels)
        df_pred = pd.concat([label_data, df],axis = 1)
        df_pred.to_csv(w_3, encoding = 'UTF-8')

        ################################################################################

        plt.cla()
        plt.clf()
        plt.scatter(test_labels, test_pred, s= 1 )
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.axis('equal')
        plt.axis('square')
        plt.grid()
        plt.xlim([0, 10])
        plt.ylim([0, 10])
        plt.plot([-1000, 1000], [-1000, 1000]);
        plt.savefig(w_4)

        ################################################################################
        if d ==1:
          plt.cla()
          plt.clf()
          error = test_pred - test_labels
          plt.hist(error, bins = 20, range = [-9,9])
          plt.xlabel("Prediction Error")
          plt.grid()
          plt.ylabel("Count")
          plt.savefig(w_5)

        elif d == 2:
          plt.cla()
          plt.clf()
          error = test_pred - test_labels
          plt.hist(error, bins = 40, range = [-19,19])
          plt.xlabel("Prediction Error")
          plt.grid()
          plt.ylabel("Count")
          plt.savefig(w_5)
        
        else:
          plt.cla()
          plt.clf()
          error = test_pred - test_labels
          plt.hist(error, bins = 40, range = [-4,4])
          plt.xlabel("Prediction Error")
          plt.grid()
          plt.ylabel("Count")
          plt.savefig(w_5)

        ################################################################################

        hist = pd.DataFrame(history.history)
        hist['epoch'] = history.epoch
        hist.to_csv(w_6, encoding = 'UTF-8')

        ################################################################################

        test_loss, test_acc = model.evaluate(normed_test_data,  test_labels, verbose=2)
        test_acc_str = str(test_acc)

        f = open(w_7,'a')
        f.write(test_acc_str)
        f.close()

        ################################################################################